In [1]:
from scipy.special import comb

def probNumOfBenign(i):
    global N,P
    global benign
    return comb(benign,i)*comb(N-benign,P-i)/comb(N,P)

def probScoreToNumOfBenign(s,i):
    global N,P,alpha,bata
    global benign
    key = (s,i)
    if key in logsScoreBenign:
        return logsScoreBenign[key]
    if (s+P)%2 != 0:
        logsScoreBenign[key] = 0
        return logsScoreBenign[key]
    m = (s+P)//2
    sum = 0
    for t in range(0,m+1):
        sum += comb(i,t)*(alpha**t)*((1-alpha)**(i-t)) * \
               comb(P-i,m-t)*(beta**(m-t))*((1-beta)**(P-i-m+t))
    logsScoreBenign[key] = sum
    return logsScoreBenign[key]
    

def _probScoreToNumOfBenign(s,i):
    global N,P,alpha
    global benign
    scores = s + P - i
    key = (s,i)
    if key in logsScoreBenign:
        return logsScoreBenign[key]
    if (scores+i)%2 != 0 or scores < -i or scores > i:
        logsScoreBenign[key] = 0
        return 0
    p = (scores+i)/2
    logsScoreBenign[key] = comb(i,p)*(alpha**p)*((1-alpha)**(i-p))

    return logsScoreBenign[key]


def probNumOfBenignToScore(i,s):
    global N,P
    global benign
    key = (i,s)
    if key in logsBenignScore:
        return logsBenignScore[key]
    if s not in logsProbBase:
        probBase = 0
        for t in range(0, P+1):
            pScoreToBengin = probScoreToNumOfBenign(s,t)
            pBenign = probNumOfBenign(t)
            probBase += pScoreToBengin*pBenign
        logsProbBase[s] = probBase
    probBase = logsProbBase[s]
    if probBase == 0:
        logsBenignScore[key] = 0
    else:
        pScoreToBengin = probScoreToNumOfBenign(s,i)
        pBenign = probNumOfBenign(i)
        logsBenignScore[key] = pScoreToBengin*pBenign/probBase
    return logsBenignScore[key]

def probGENumOfBenignToScores(i,s):
    global P
    sum = 0
    for j in range(i,P+1):
        sum += probNumOfBenignToScore(j,s)
    return sum

def probGNumOfBenignToScores(i,s):
    global P
    sum = 0
    for j in range(i+1,P+1):
        sum += probNumOfBenignToScore(j,s)
    return sum

def probLENumOfBenignToScores(i,s):
    sum = 0
    for j in range(0,i+1):
        sum += probNumOfBenignToScore(j,s)
    return sum

def probLNumOfBenignToScores(i,s):
    sum = 0
    for j in range(0,i):
        sum += probNumOfBenignToScore(j,s)
    return sum

In [2]:
if __name__ == '__main__':
    logsScoreBenign = {}
    logsBenignScore = {}
    logsProbBase = {}
    benign = 3
    N = 5
    P = 3
    s = -8
    alpha = 0.5
    beta = 0.5
    
    half = int(P / 2 + 1)
    for i in range(0,P):
        print(f'Prob(S=3|B={i})={probScoreToNumOfBenign(3,i)},Prob(B={i})={probNumOfBenign(i)}')
    print(f'Prob(S=3|B={3})={probScoreToNumOfBenign(3,3)},Prob(B={3})={probNumOfBenign(3)}')
    print(f'N={N}, P={P},Benign={benign},alpha={alpha},beta={beta}')
    print(f'Quorum={half}')
    print("ErrPunish\t\t\t ErrIncent")
    for s in range(0,P+1):
        # Malicious < Quorum -> P-Benign < Quorum -> Benign > P-Quorum
        pErrPunish = probGNumOfBenignToScores(P-half,-s)
        # Benign < Quorum
        pErrIncent = probLNumOfBenignToScores(half,s)
        if pErrPunish == 0 or pErrIncent == 0:
            continue
        print('Prob(Malicious <%3d|S =%3d)=%.3f, Prob(Benign <%3d|S =%3d)=%.3f' % (half,-s,pErrPunish, half,s,pErrIncent))
    


Prob(S=3|B=0)=0.125,Prob(B=0)=0.0
Prob(S=3|B=1)=0.125,Prob(B=1)=0.3
Prob(S=3|B=2)=0.125,Prob(B=2)=0.6
Prob(S=3|B=3)=0.125,Prob(B=3)=0.1
N=5, P=3,Benign=3,alpha=0.5,beta=0.5
Quorum=2
ErrPunish			 ErrIncent
Prob(Malicious <  2|S = -1)=0.700, Prob(Benign <  2|S =  1)=0.300
Prob(Malicious <  2|S = -3)=0.700, Prob(Benign <  2|S =  3)=0.300


In [3]:
probLNumOfBenignToScores(2,1),probNumOfBenignToScore(2,1),probGNumOfBenignToScores(2,1)

(0.3, 0.6, 0.10000000000000002)

In [6]:
comb(3,5)

0.0